In [0]:
%sql
show tables

database,tableName,isTemporary
default,bronze_amazon_sales,false
default,employees,false
default,gold_dim_products,false
default,gold_dim_reviews,false
default,gold_dim_users,false
default,gold_fact_product_performance,false
default,silver_amazon_base,false
default,silver_amazon_sales,false
default,silver_dim_products,false
default,silver_dim_reviews,false


In [0]:
df_bronze = spark.table("bronze_amazon_sales")

df_bronze.printSchema()
df_bronze.show(10, truncate=False)
display(df_bronze.limit(10))


root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- discounted_price: string (nullable = true)
 |-- actual_price: string (nullable = true)
 |-- discount_percentage: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_count: string (nullable = true)
 |-- about_product: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- review_content: string (nullable = true)
 |-- img_link: string (nullable = true)
 |-- product_link: string (nullable = true)

+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------+---------

product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
B07JW9H4J1,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,₹399,"₹1,099",64%,4.2,"24,269","High Compatibility : Compatible With iPhone 12, 11, X/XsMax/Xr ,iPhone 8/8 Plus,iPhone 7/7 Plus,iPhone 6s/6s Plus,iPhone 6/6 Plus,iPhone 5/5s/5c/se,iPad Pro,iPad Air 1/2,iPad mini 1/2/3,iPod nano7,iPod touch and more apple devices.|Fast Charge&Data Sync : It can charge and sync simultaneously at a rapid speed, Compatible with any charging adaptor, multi-port charging station or power bank.|Durability : Durable nylon braided design with premium aluminum housing and toughened nylon fiber wound tightly around the cord lending it superior durability and adding a bit to its flexibility.|High Security Level : It is designed to fully protect your device from damaging excessive current.Copper core thick+Multilayer shielding, Anti-interference, Protective circuit equipment.|WARRANTY: 12 months warranty and friendly customer services, ensures the long-time enjoyment of your purchase. If you meet any question or problem, please don't hesitate to contact us.","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBBSNLYT3ONILA,AHCTC6ULH4XB6YHDY6PCH2R772LQ,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,AENGU523SXMOS7JPDTW52PNNVWGQ,AEQJHCVTNINBS4FKTBGQRQTGTE5Q,AFC3FFC5PKFF5PMA52S3VCHOZ5FQ","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jaspreet singh,Khaja moin,Anand,S.ARUMUGAM","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1KD19VHEDV0OR,R3C02RMYQMK6FC,R39GQRVBUZBWGY,R2K9EDOE15QIRJ,R3OI7YT648TL8I","Satisfied,Charging is really fast,Value for money,Product review,Good quality,Good product,Good Product,As of now seems good","Looks durable Charging is fine tooNo complains,Charging is really fast, good product.,Till now satisfied with the quality.,This is a good product . The charging speed is slower than the original iPhone cable,Good quality, would recommend,https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/81---F1ZgHL._SY88.jpg,Product had worked well till date and was having no issue.Cable is also sturdy enough...Have asked for replacement and company is doing the same...,Value for money",https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/51UsScvHQNL._SX300_SY300_QL70_FMwebp_.jpg,https://www.amazon.in/Wayona-Braided-WN3LG1-Syncing-Charging/dp/B07JW9H4J1/ref=sr_1_1?qid=1672909124&s=electronics&sr=1-1
B098NS6PVG,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be it an android smartphone (Mi, Samsung, Oppo, Vivo, Realme, OnePlus, etc), tablet, laptop (Macbook, Chromebook, etc)|Supports Quick Charging (2.0/3.0)|Unbreakable – Made of special braided outer with rugged interior bindings, it is ultra-durable cable that won’t be affected by daily rough usage|Ideal Length – It has ideal length of 1.5 meters which is neither too short like your typical 1meter cable or too long like a 2meters cable|Supports maximum 3A fast charging and 480 Mbps data transfer speed|6 months manufacturer warranty from the date of purchase","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBXNGXZJT525AQ,AHONIZU3ICIEHQIGQ6R2VFRSBXOQ,AFPHD2CRPDZMWMBL7WXRSVYWS5JA,AEZ346GX3HJ4O4XNRPHCNHXQURMQ,AEPSWFPNECKO34PUC7I56ITGXR6Q,AHWVEHR5DYLVFTO2KF3IZATFQSWQ,AH4QT33M55677I7ISQOAKEQWACYQ","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Placeholder,BharanI,sonia,Niam","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GW

# Standardize Columns

In [0]:
from pyspark.sql.functions import col, regexp_replace, when

df_silver = df_bronze.select(
    col("product_id"),
    col("product_name"),
    col("category"),
    when(
        regexp_replace(col("discounted_price"), "[^0-9]", "").rlike(r"^\d+$"),
        regexp_replace(col("discounted_price"), "[^0-9]", "").cast("int")
    ).otherwise(None).alias("discounted_price"),
    when(
        regexp_replace(col("actual_price"), "[^0-9]", "").rlike(r"^\d+$"),
        regexp_replace(col("actual_price"), "[^0-9]", "").cast("int")
    ).otherwise(None).alias("actual_price"),
    when(
        regexp_replace(col("discount_percentage"), "[^0-9]", "").rlike(r"^\d+$"),
        regexp_replace(col("discount_percentage"), "[^0-9]", "").cast("int")
    ).otherwise(None).alias("discount_percentage"),
    when(
        regexp_replace(col("rating"), "[^0-9.]", "").rlike(r"^\d*\.?\d+$"),
        regexp_replace(col("rating"), "[^0-9.]", "").cast("double")
    ).otherwise(None).alias("rating"),
    when(
        regexp_replace(col("rating_count"), "[^0-9]", "").rlike(r"^\d+$"),
        regexp_replace(col("rating_count"), "[^0-9]", "").cast("int")
    ).otherwise(None).alias("rating_count"),
    col("about_product"),
    col("user_id"),
    col("user_name"),
    col("review_id"),
    col("review_title"),
    col("review_content"),
    col("img_link"),
    col("product_link")
)

In [0]:
from pyspark.sql.functions import col, regexp_replace, when
import re

def clean_and_cast(column):
    cleaned = regexp_replace(column, "[^0-9.]", "")
    # Only cast if cleaned value matches a valid float pattern
    return when((cleaned == "") | (~cleaned.rlike(r"^\\d*\\.?\\d+$")), None).otherwise(cleaned.cast("double"))

df_silver = df_bronze.select(
    col("product_id"),
    col("product_name"),
    col("category"),
    clean_and_cast("discounted_price").alias("discounted_price"),
    clean_and_cast("actual_price").alias("actual_price"),
    clean_and_cast("discount_percentage").alias("discount_percentage"),
    col("rating"),
    col("rating_count"),
    col("about_product"),
    col("user_id"),
    col("user_name"),
    col("review_id"),
    col("review_title"),
    col("review_content"),
    col("img_link"),
    col("product_link")
)

In [0]:
display(df_silver.limit(10))

product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
B07JW9H4J1,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,null,null,null,4.2,"24,269","High Compatibility : Compatible With iPhone 12, 11, X/XsMax/Xr ,iPhone 8/8 Plus,iPhone 7/7 Plus,iPhone 6s/6s Plus,iPhone 6/6 Plus,iPhone 5/5s/5c/se,iPad Pro,iPad Air 1/2,iPad mini 1/2/3,iPod nano7,iPod touch and more apple devices.|Fast Charge&Data Sync : It can charge and sync simultaneously at a rapid speed, Compatible with any charging adaptor, multi-port charging station or power bank.|Durability : Durable nylon braided design with premium aluminum housing and toughened nylon fiber wound tightly around the cord lending it superior durability and adding a bit to its flexibility.|High Security Level : It is designed to fully protect your device from damaging excessive current.Copper core thick+Multilayer shielding, Anti-interference, Protective circuit equipment.|WARRANTY: 12 months warranty and friendly customer services, ensures the long-time enjoyment of your purchase. If you meet any question or problem, please don't hesitate to contact us.","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBBSNLYT3ONILA,AHCTC6ULH4XB6YHDY6PCH2R772LQ,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,AENGU523SXMOS7JPDTW52PNNVWGQ,AEQJHCVTNINBS4FKTBGQRQTGTE5Q,AFC3FFC5PKFF5PMA52S3VCHOZ5FQ","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jaspreet singh,Khaja moin,Anand,S.ARUMUGAM","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1KD19VHEDV0OR,R3C02RMYQMK6FC,R39GQRVBUZBWGY,R2K9EDOE15QIRJ,R3OI7YT648TL8I","Satisfied,Charging is really fast,Value for money,Product review,Good quality,Good product,Good Product,As of now seems good","Looks durable Charging is fine tooNo complains,Charging is really fast, good product.,Till now satisfied with the quality.,This is a good product . The charging speed is slower than the original iPhone cable,Good quality, would recommend,https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/81---F1ZgHL._SY88.jpg,Product had worked well till date and was having no issue.Cable is also sturdy enough...Have asked for replacement and company is doing the same...,Value for money",https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/51UsScvHQNL._SX300_SY300_QL70_FMwebp_.jpg,https://www.amazon.in/Wayona-Braided-WN3LG1-Syncing-Charging/dp/B07JW9H4J1/ref=sr_1_1?qid=1672909124&s=electronics&sr=1-1
B098NS6PVG,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,null,null,null,4.0,"43,994","Compatible with all Type C enabled devices, be it an android smartphone (Mi, Samsung, Oppo, Vivo, Realme, OnePlus, etc), tablet, laptop (Macbook, Chromebook, etc)|Supports Quick Charging (2.0/3.0)|Unbreakable – Made of special braided outer with rugged interior bindings, it is ultra-durable cable that won’t be affected by daily rough usage|Ideal Length – It has ideal length of 1.5 meters which is neither too short like your typical 1meter cable or too long like a 2meters cable|Supports maximum 3A fast charging and 480 Mbps data transfer speed|6 months manufacturer warranty from the date of purchase","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBXNGXZJT525AQ,AHONIZU3ICIEHQIGQ6R2VFRSBXOQ,AFPHD2CRPDZMWMBL7WXRSVYWS5JA,AEZ346GX3HJ4O4XNRPHCNHXQURMQ,AEPSWFPNECKO34PUC7I56ITGXR6Q,AHWVEHR5DYLVFTO2KF3IZATFQSWQ,AH4QT33M55677I7ISQOAKEQWACYQ","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Placeholder,BharanI,sonia,Niam","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE

### Remove Bad Records

In [0]:
df_silver = df_silver.filter(col("product_id").isNotNull())

# Save Silver Base Table

In [0]:
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_amazon_base")


## Create Silver DIMENSION Tables

## Product Dimension

In [0]:
%sql
drop table if exists `gold_dim_reviews`

In [0]:
df_products = df_silver.select(
    "product_id",
    "product_name",
    "category",
    "actual_price",
    "discounted_price",
    "discount_percentage",
    "rating",
    "rating_count",
    "about_product",
    "img_link",
    "product_link"
).dropDuplicates()

df_products.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_dim_products")


## User Dimension

In [0]:
df_users = df_silver.select(
    "user_id",
    "user_name"
).dropDuplicates()

df_users.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_dim_users")


## Review Dimension

In [0]:
df_reviews = df_silver.select(
    "review_id",
    "review_title",
    "review_content",
    "product_id",
    "user_id"
).dropDuplicates()

df_reviews.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_dim_reviews")


# GOLD Layer – FACT TABLE (CORE ANALYTICS)
### # Fact = Product Review & Pricing Performance

## Fact Product Performance

In [0]:
df_fact_product = df_silver.select(
    "product_id",
    "user_id",
    "review_id",
    "actual_price",
    "discounted_price",
    "discount_percentage",
    "rating",
    "rating_count"
)


## Add Derived Metrics

In [0]:
from pyspark.sql.functions import (col)

df_fact_product = df_fact_product.withColumn(
    "discount_amount",
    col("actual_price") - col("discounted_price")
)


## Save Gold Fact Table

In [0]:
df_fact_product.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_fact_product_performance")


## GOLD Dimension Tables

In [0]:
spark.table("silver_dim_products") \
    .write.format("delta").mode("overwrite") \
    .saveAsTable("gold_dim_products")

spark.table("silver_dim_users") \
    .write.format("delta").mode("overwrite") \
    .saveAsTable("gold_dim_users")

spark.table("silver_dim_reviews") \
    .write.format("delta").mode("overwrite") \
    .saveAsTable("gold_dim_reviews")

In [0]:
spark.table("gold_fact_product_performance").count()


1465

In [0]:
spark.table("gold_dim_products").count()


1456

In [0]:
spark.table("gold_dim_users").count()


1206

In [0]:
spark.table("gold_dim_reviews").count()

1369

In [0]:
spark.sql("""
SELECT 
  COUNT(DISTINCT product_id) AS total_products,
  AVG(try_cast(rating AS DOUBLE)) AS avg_rating
FROM gold_fact_product_performance
""").show()


+--------------+-----------------+
|total_products|       avg_rating|
+--------------+-----------------+
|          1351|4.097299218194754|
+--------------+-----------------+



In [0]:
spark.sql("""
SELECT 
  ROUND(AVG(try_cast(rating AS DOUBLE)), 2) AS avg_rating
FROM gold_fact_product_performance
""").show()


+----------+
|avg_rating|
+----------+
|       4.1|
+----------+

